# Setting up your own MediaWiki

> Uses Containers and ACR

## Point to the right account

In [ ]:
az account set -s fcda8dd1-d933-4405-8d6e-c6b573500a52

In [ ]:
az account show

## Create the Resource Group

> Everything should go in here.  I am not using PowerShell variables, like I should.  This is down and dirty.

In [ ]:
az group create --name HomolWikiRG --location eastus

## Create the Azure Container Registry to store whatever images we will need for this

In [ ]:
az acr create --name HomolWikiACR --location eastus --resource-group HomolWikiRG --sku Basic

## Import/Add MediaWiki server

In [ ]:
az acr import -n HomolWikiACR --source docker.io/library/mediawiki:latest

> IMPORTANT: Turn on Admin Keys on the ACR FIRST

## Add Azure storage as a volume to the container.  

> Hoping that this will allow us to transfer files, like the localSettings.php file

In [ ]:
az storage account create --resource-group HomolWikiRG --location eastus --name homolwikistorage --sku STANDARD_LRS

## Create the FileShare

In [ ]:
az storage share create --name acishare --account-name homolwikistorage

In [ ]:
az storage account keys list --resource-group HomolWikiRG --account-name homolwikistorage --query "[0].value" --output tsv

## Create the actual web server

> This attaches the FileShare for saving state (if possible) and transfering files.  It also pulls the image from ACR and provides the domain prefix of homolwiki

In [ ]:
az container create -g HomolWikiRG -n homolwiki --image homolwikiacr.azurecr.io/library/mediawiki:latest --ip-address Public --ports 80 443 22 --registry-username HomolWikiACR --registry-password YH5CBPMBeu=C3sGd58uB4v0qyeZVY2JL --dns-name-label homolwiki --azure-file-volume-account-name homolwikistorage --azure-file-volume-account-key J2+lccnZEhQe76Q0E553BQNZlSUZcq736pgdy5NbsNjzZQYPJ6wCJLYOYkTUbXnV+aLmUof8ad/IVrUHPj4YCw== --azure-file-volume-share-name acishare --azure-file-volume-mount-path /aci/

## Import/Add MySQL server

> This step may not be necessary for testing, now that we know that you can just choose SQLite to run everything directly from a single container.  I was never able to get the site container to talk to the DB contaner.

In [ ]:
az acr import -n HomolWikiACR --source docker.io/library/mysql:latest

In [ ]:
az container create -g HomolWikiRG -n homolwikidb --image homolwikiacr.azurecr.io/library/mysql:latest --ip-address Public -e MYSQL_ROOT_PASSWORD=  --ports 80 8080

## Setup your MediaWiki site.

> If all went well, you should have a site at http://homolwiki.eastus.azurecontainer.io

> Walk through the wizard.  If you skipped the MySQL portion, be sure to select the SQLite option during setup. At the end you should get a `localsettings.php` file

## Add the file to the mounted FileShare drive

> Download the `localSettings.php` file and upload it to a location inside the new file share volume you made in Azure Storage.  I made a new directory called wiki_install and uploaded the file there, which made the file path this: https://homolwikistorage.file.core.windows.net/acishare/wiki_install/LocalSettings.php

## Bash

In [ ]:
az container exec --resource-group HomolWikiRG --name homolwiki --exec-command "/bin/bash"

> This gets you a bash terminal on the running container.  Woo hoo!

> Here is the command you need to run on that terminal session:

```
cp /aci/wiki_install/LocalSettings.php .
```

> Now `localSettings.php` should exist in the site's root directory.  Run `ls` to confirm you see it.

## All set!

> Your wiki should now show up, as expected.